In [ ]:
# %pip install optuna seqeval evaluate -q

In [1]:
import pandas as pd
import ast
from tqdm import tqdm
from collections import Counter
import datasets
from datasets import (Dataset, Features, Sequence, Value, ClassLabel, load_dataset,
                    load_from_disk, concatenate_datasets, DatasetDict)
from sklearn.model_selection import KFold
from transformers import (AutoTokenizer, AutoModel, AutoModelForTokenClassification,
                         pipeline, TrainingArguments, Trainer,
                         DataCollatorForTokenClassification, EarlyStoppingCallback, BertForMaskedLM,
                        AutoConfig, AutoModelForMaskedLM)
import torch
import optuna
from optuna.pruners import MedianPruner
import os
os.environ['WANDB_DISABLED'] = 'true'
import pickle
import numpy as np
import seqeval
import evaluate
from seqeval.metrics import classification_report, precision_score, recall_score, f1_score, accuracy_score

seqeval = evaluate.load("seqeval")

/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2025-10-02 07:50:53.208683: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-02 07:50:53.257393: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-02 07:50:54.181737: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Загрузка

In [3]:
TRAIN_PATH = "/home/jupyter/datasphere/project/mikhail/train_with_augmented_volume_percent.csv"
SUBMISSION_PATH = "/home/jupyter/datasphere/project/mikhail/submission.csv"

In [4]:
train_df_raw = pd.read_csv(TRAIN_PATH, sep=';')
train_df_raw

,sample,annotation
0,aa,"[(0, 2, 'O')]"
1,aala,"[(0, 4, 'O')]"
2,aarcca,"[(0, 6, 'O')]"
3,abon,"[(0, 4, 'O')]"
4,abso,"[(0, 4, 'B-BRAND')]"
...,...,...
27547,ветчина 300 гр нарезка,"[(0, 7, 'B-TYPE'), (8, 11, 'B-VOLUME'), (12, 1..."
27548,кукуруза 400 г mikado,"[(0, 8, 'B-TYPE'), (9, 12, 'B-VOLUME'), (13, 1..."
27549,кукуруза 340 гр,"[(0, 8, 'B-TYPE'), (9, 12, 'B-VOLUME'), (13, 1..."
27550,хлеб 350 г 5 злаков,"[(0, 4, 'B-TYPE'), (5, 8, 'B-VOLUME'), (9, 10,..."


In [5]:
train_df_raw['annotation'] = train_df_raw['annotation'].str.replace("\'0\'", "O")

In [6]:
train_df_raw['annotation'] = train_df_raw['annotation'].apply(lambda x: ast.literal_eval(str(x)))

In [7]:
all_words = []
all_tags = []
for i, row in tqdm(train_df_raw.iterrows()):
    words_sample = []
    entities_sample = []
    for ent in row['annotation']:
        word = row['sample'][ent[0]:ent[1]]
        words_sample.append(word)
        entities_sample.append(ent[2])
    all_words.append(words_sample)
    all_tags.append(entities_sample)
    # print(words_sample, entities_sample)

27552it [00:01, 18988.87it/s]


In [8]:
lbls_in_dataset = [
 'O',
 'B-BRAND',
 'B-PERCENT',
 'B-TYPE',
 'B-VOLUME',
 'I-BRAND',
 'I-PERCENT',
 'I-TYPE',
 'I-VOLUME']
label2id = {v:i for i, v in enumerate(lbls_in_dataset)}
id2label = {i:v for i, v in enumerate(lbls_in_dataset)}

In [9]:
features=Features(
    {
        "id": Value(dtype='int32', id=None),
        "tokens": Sequence(feature=Value(dtype='string', id=None)),
        "ner_tags": Sequence(feature=ClassLabel(num_classes=len(lbls_in_dataset), names=list(lbls_in_dataset)), id=None)
    }
)

In [10]:
ds = Dataset.from_dict({"id": list(range(len(all_words))),
                        "tokens": all_words,
                        "ner_tags": all_tags},
                       features=features)

In [11]:
ds_splitted = ds.train_test_split(
    test_size=0.2, shuffle=True, seed=42,
    # stratify_by_column='ner_tags'
)

In [12]:
model_path = "Dersty/distilrubert_X5_ner_MLM"

In [13]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [14]:
tokenizer = AutoTokenizer.from_pretrained(model_path,
                                          use_fast=True,
                                          add_prefix_space=True)

model = AutoModelForTokenClassification.from_pretrained(
    model_path,
    num_labels=len(lbls_in_dataset),
    id2label=id2label,
    label2id=label2id,
).to(device)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at Dersty/distilrubert_X5_ner_MLM and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Подготовка данных

In [15]:
def tokenize_and_align_labels(examples, tokenizer):
    tokenized_inputs = tokenizer(examples['tokens'], truncation=True, is_split_into_words=True,
        padding="max_length",   
        max_length=16, 
                                )

    labels = []
    for i, label in enumerate(examples['ner_tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_index = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            else:
                label_ids.append(label[word_idx])
        labels.append(label_ids)

    tokenized_inputs['labels'] = labels
    return tokenized_inputs

In [16]:
tokenized_ds_train = ds_splitted['train'].map(tokenize_and_align_labels,
                                              batched=True,
                                              fn_kwargs={'tokenizer': tokenizer})

Map: 100%|██████████| 22041/22041 [00:01<00:00, 17073.30 examples/s]


In [17]:
tokenized_ds_test = ds_splitted['test'].map(tokenize_and_align_labels,
                                              batched=True,
                                              fn_kwargs={'tokenizer': tokenizer})

Map: 100%|██████████| 5511/5511 [00:00<00:00, 21783.94 examples/s]


In [18]:
data_collator = DataCollatorForTokenClassification(tokenizer)

## Поиск гиперпараметров с помощью OPTUNA

In [19]:
# функция метрики
def compute_metrics_custom(p):
    labels_list = list(id2label.values())

    if hasattr(p, 'predictions'):
        predictions = p.predictions
        labels = p.label_ids
    else:
        predictions, labels = p
        
    print(f"DEBUG: predictions type: {type(predictions)}, shape: {getattr(predictions, 'shape', 'No shape')}")
    print(f"DEBUG: labels type: {type(labels)}, shape: {getattr(labels, 'shape', 'No shape')}")
    

    if isinstance(predictions, tuple):
        predictions = predictions[0]  # Take logits
    
    # Convert to numpy if they're tensors
    if hasattr(predictions, 'cpu'):
        predictions = predictions.cpu().numpy()
    if hasattr(labels, 'cpu'):
        labels = labels.cpu().numpy()
    
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [labels_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [labels_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    report_dict = classification_report(true_labels, true_predictions, digits=4, output_dict=True)
    report = classification_report(true_labels, true_predictions, digits=4)
    macro_f1 = report_dict["macro avg"]["f1-score"]
    print("=== seqeval classification_report ===")
    print(report)
    CLASS_REPORT_PATH = 'logs/last_classification_report2.txt'
    try:
        with open(CLASS_REPORT_PATH, "w", encoding="utf-8") as f:
            f.write(report)
    except Exception as e:
        print(f"Warning: failed to write classification report to {CLASS_REPORT_PATH}: {e}")

    return {
        "f1_macro": macro_f1,
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [20]:
early_stop_cb = EarlyStoppingCallback(early_stopping_patience=2)

In [21]:
def objective(trial: optuna.Trial):
    print(f"\n=== Trial {trial.number} ===")
    print(trial.params)
    
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 5e-4, log=True)
    weight_decay = trial.suggest_float('weight_decay', 0.001, 0.1, log=True)
    num_train_epochs = trial.suggest_int('num_train_epochs', 4, 14)

    print(f"\n=== Current parameters: {trial.params} ===")
    

    model = AutoModelForTokenClassification.from_pretrained(
        model_path, 
        num_labels=len(lbls_in_dataset),
        id2label=id2label,
        label2id=label2id
    ).to(device)
    print(model_path)

    trial_check_dir = f"./distilrubert_X5_ner_MLM_ft_0210/checkpoints_trial_{trial.number}"
    os.makedirs(trial_check_dir, exist_ok=True)

    args = TrainingArguments(
        output_dir=trial_check_dir,
        overwrite_output_dir=True,
        eval_strategy="epoch",
        torch_compile=False, # Для Яндекса Датасферы
        per_device_train_batch_size=1024,   
        per_device_eval_batch_size=512,
        learning_rate=learning_rate,
        weight_decay=weight_decay,
        num_train_epochs=num_train_epochs,
        seed=42     ,          
        data_seed=24,
        gradient_accumulation_steps=1,   
        
        lr_scheduler_type="cosine",
        
        warmup_ratio=0.1,
        report_to=None,
        logging_dir="./logs",
        logging_steps=100,
        load_best_model_at_end=False,
        metric_for_best_model="eval_f1_macro",
        save_total_limit=1,
        save_strategy="no",
        disable_tqdm=False,
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=tokenized_ds_train,
        eval_dataset=tokenized_ds_test,
        data_collator=data_collator,
        compute_metrics=compute_metrics_custom,
        tokenizer=tokenizer,
        callbacks=[early_stop_cb] if 'early_stop_cb' in globals() else [],
    )

    trainer.train()

    eval_metrics = trainer.evaluate()
    f1_score = eval_metrics["eval_f1_macro"]

    trial.report(f1_score, step=0)

    if trial.should_prune():
        del model, trainer
        torch.cuda.empty_cache()
        raise optuna.TrialPruned()

    print(f"✅ Trial {trial.number} finished | F1: {f1_score:.4f}")
    return f1_score


study = optuna.create_study(
    study_name='test_optuna',
    direction='maximize',
    pruner=MedianPruner(
        n_startup_trials=2,
        n_warmup_steps=1,     
        interval_steps=1     
    )
)

study.optimize(objective, n_trials=25)

print("Best trial:")
print(f"  Value: {study.best_value:.4f}")
print("  Params:")
for key, value in study.best_params.items():
    print(f"    {key}: {value}")

[I 2025-10-02 07:51:31,474] A new study created in memory with name: test_optuna



=== Trial 0 ===
{}

=== Current parameters: {'learning_rate': 0.00011410644661724481, 'weight_decay': 0.007328327054148027, 'num_train_epochs': 14} ===


Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at Dersty/distilrubert_X5_ner_MLM and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipykernel_11715/1166984652.py:74: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Dersty/distilrubert_X5_ner_MLM


Using EarlyStoppingCallback without load_best_model_at_end=True. Once training is finished, the best model will not be loaded automatically.
100%|██████████| 11/11 [00:02<00:00,  5.04it/s]

...

=== seqeval classification_report ===
              precision    recall  f1-score   support

       BRAND     0.9100    0.9255    0.9177      3475
     PERCENT     0.9529    0.9759    0.9643        83
        TYPE     0.9601    0.9706    0.9653     10485
      VOLUME     0.9592    0.9038    0.9307        52

   micro avg     0.9476    0.9593    0.9534     14095
   macro avg     0.9456    0.9440    0.9445     14095
weighted avg     0.9477    0.9593    0.9534     14095

{'eval_loss': 0.24248403310775757, 'eval_f1_macro': 0.9444967148901053, 'eval_precision': 0.9476450798990749, 'eval_recall': 0.9592763391273501, 'eval_f1': 0.9534252371046786, 'eval_accuracy': 0.94567014423351, 'eval_runtime': 4.8304, 'eval_samples_per_second': 1140.907, 'eval_steps_per_second': 2.277, 'epoch': 5.0}
{'train_runtime': 151.05, 'train_samples_per_second': 1313.267, 'train_steps_per_second': 1.311, 'train_loss': 0.2952461768280376, 'epoch': 5.0}



100%|██████████| 11/11 [00:02<00:00,  4.95it/s]

DEBUG: predictions type: <class 'tuple'>, shape: No shape
DEBUG: labels type: <class 'numpy.ndarray'>, shape: (5511, 16)


100%|██████████| 11/11 [00:04<00:00,  2.27it/s]
[I 2025-10-02 09:12:49,738] Trial 24 finished with value: 0.9444967148901053 and parameters: {'learning_rate': 0.00025187860901377916, 'weight_decay': 0.010573122833122712, 'num_train_epochs': 9}. Best is trial 6 with value: 0.9512251951099278.


=== seqeval classification_report ===
              precision    recall  f1-score   support

       BRAND     0.9100    0.9255    0.9177      3475
     PERCENT     0.9529    0.9759    0.9643        83
        TYPE     0.9601    0.9706    0.9653     10485
      VOLUME     0.9592    0.9038    0.9307        52

   micro avg     0.9476    0.9593    0.9534     14095
   macro avg     0.9456    0.9440    0.9445     14095
weighted avg     0.9477    0.9593    0.9534     14095

✅ Trial 24 finished | F1: 0.9445
Best trial:
  Value: 0.9512
  Params:
    learning_rate: 0.0004026606979829003
    weight_decay: 0.008132177642386032
    num_train_epochs: 6


## Обучаем новую модель

In [31]:
tokenizer = AutoTokenizer.from_pretrained(model_path,
                                          use_fast=True,
                                          add_prefix_space=True)

model = AutoModelForTokenClassification.from_pretrained(
    model_path,
    num_labels=len(lbls_in_dataset),
    id2label=id2label,
    label2id=label2id,
).to(device)

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 8de2b58d-6a8b-42ea-b30b-c54abb53e1cb)')' thrown while requesting HEAD https://huggingface.co/Dersty/distilrubert_X5_ner_MLM/resolve/main/tokenizer_config.json
Retrying in 1s [Retry 1/5].
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at Dersty/distilrubert_X5_ner_MLM and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:

ds_tokenized = ds.map(tokenize_and_align_labels,
              batched=True,
              fn_kwargs={'tokenizer': tokenizer})

training_args = TrainingArguments(
    eval_strategy="no", 
    torch_compile=False, # попробую отключить для Я Датасферы
    per_device_train_batch_size=1024,
    learning_rate=0.0004026606979829003,      
    weight_decay= 0.008132177642386032,
    num_train_epochs=6,        
    
    lr_scheduler_type="cosine",
    
    
    seed=42,
    data_seed=24,
    gradient_accumulation_steps=1,
    warmup_ratio=0.1,
    report_to=None,
    logging_dir="./logs",
    logging_steps=20,
    )

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_tokenized,
    data_collator=data_collator,
    tokenizer=tokenizer,

)

print("Starting training without evaluation...")
trainer.train()
trained_model = trainer.model

Map: 100%|██████████| 27552/27552 [00:01<00:00, 23378.41 examples/s]
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipykernel_11715/3574406872.py:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Starting training without evaluation...


 12%|█▏        | 20/162 [00:22<02:38,  1.12s/it]

{'loss': 0.8937, 'grad_norm': 2.991623878479004, 'learning_rate': 0.00040247170963608265, 'epoch': 0.74}


 25%|██▍       | 40/162 [00:44<02:17,  1.12s/it]

...

{'loss': 0.022, 'grad_norm': 0.25954264402389526, 'learning_rate': 2.448458977917428e-05, 'epoch': 5.19}


 99%|█████████▉| 160/162 [03:02<00:02,  1.17s/it]

{'loss': 0.016, 'grad_norm': 0.22310884296894073, 'learning_rate': 4.2514061360391687e-07, 'epoch': 5.93}


100%|██████████| 162/162 [03:42<00:00,  1.37s/it]

{'train_runtime': 222.6735, 'train_samples_per_second': 742.396, 'train_steps_per_second': 0.728, 'train_loss': 0.18602542599870098, 'epoch': 6.0}


## Сохраняем дообученную версию

In [52]:
model.save_pretrained("./distilrubert_optuna_ft_09344")
tokenizer.save_pretrained("./distilrubert_optuna_ft_09344")

('./distilrubert_optuna_ft_09344/tokenizer_config.json',
 './distilrubert_optuna_ft_09344/special_tokens_map.json',
 './distilrubert_optuna_ft_09344/vocab.txt',
 './distilrubert_optuna_ft_09344/added_tokens.json',
 './distilrubert_optuna_ft_09344/tokenizer.json')